# 데이터 불러오기

In [12]:
ver = 1
k_list = [10] # [7,8,9,10,11,12,13]
umap = 3
epoch = 300

In [13]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings; 
warnings.filterwarnings("always"); 
warnings.filterwarnings(action='ignore')

data_dir = './data/'

df = pd.read_feather(data_dir + 'clustering_data_1901_2004_ver{0}.ftr'.format(ver))
info_features = df.loc[:, :'지역'].columns.tolist()
features = df.loc[:, '가정용품':'주거'].columns.tolist()

In [17]:
all_data_k = []
for k in k_list:
  pred = pd.read_csv(
      data_dir + 'clustering_results/ver{0}/pred_cluster{1}_umap{2}_epochs{3}_batchsize64.csv'.format(ver, k, umap, epoch))
  df['cluster'] = pred['0'].values

  # 클러스터 관측 수 많은 순으로 정렬
  as_is_list = df.cluster.value_counts().keys()
  to_be_list = range(1, k+1)
  replacing_dict = dict(zip(as_is_list, to_be_list))
  df['cluster'] = df['cluster'].replace(replacing_dict) 

  all_data_k.append(df)


# 실루엣 점수
- 최대 점수: 1
- 실제로 잘 동작하지는 않고, 군집에 어떤 의미의 데이터가 있는지 그리고 클러스터가 사용자 기대에 부합하는지 알려면 클러스터를 직접 확인해야 함

In [ ]:
# k_list = [7,8,9,10,11,12,13]  
# score_list = [0.039, 0.025, -0.001, 0.084, 0.099, 0.084, 0.102]
# 4시간 30분

from sklearn.metrics import silhouette_score # 4시간30분

for data in tqdm(all_data_k):

  X = data.iloc[:, 7:25]
  y = data['cluster']

  print(f'Silhouette Score: {silhouette_score(X, y):.3f}')


 14%|█▍        | 1/7 [28:51<2:53:07, 1731.30s/it]

Silhouette Score: 0.039


 29%|██▊       | 2/7 [57:41<2:24:14, 1730.88s/it]

Silhouette Score: 0.025


 43%|████▎     | 3/7 [1:26:21<1:55:02, 1725.72s/it]

...

Silhouette Score: 0.099


 86%|████████▌ | 6/7 [2:52:33<28:43, 1723.36s/it]

Silhouette Score: 0.084


100%|██████████| 7/7 [3:21:13<00:00, 1724.83s/it]

Silhouette Score: 0.102


# (table) count & feature vector
- k_list 7개 : 20분 정도

## 월별 이동 카운트
- 분석 대상 그룹: 성별 2개(남,여), 나이 5개(20, 30, 40, 50, 60+), 소득 11개(1~11), 지역207개 = 22770개
- 각 그룹은 월별로 속하는 클러스터 label 가짐. 이것이 총 몇 번 바꼈는지 카운트.
  - 클러스터 개수 늘렸을 때, 카운트가 적게 증가하는 구간 찾기 (e.g., 클러스터 개수 7→8 늘렸는데 카운트가 그대로였다면, 8이 괜찮지 않을까 생각)
- 그룹 수가 많아서, 중간에 관측 끊기는 경우도 있음
- [count가 적게 증가했다] 의미: 클러스터링이 이미 잘 되어 있어서 k를 늘려도 membership이 거의 유지되었다 

In [ ]:
count_list = []

for data in all_data_k:

  df = data.drop(data.columns[7:25],axis=1)
  k = len(df.cluster.unique())

  df = df.sort_values(by=['년','월'])
  grouped = df.groupby(by=['가맹점소재지1','가맹점소재지2','성별','연령대별','연평균소득추정'])

  transition_dict = {}
  for key in grouped.groups.keys():
    transition_dict[key] = list(grouped.get_group(key)['cluster'])

  """ 카운트 """

  count = 0

  for key,item in transition_dict.items():
    count += np.count_nonzero(np.diff(item))

  print(f'move count (k = {k}): ', count)

  count_list.append(count)

move count (k = 7):  113328
move count (k = 8):  119760
move count (k = 9):  126871
move count (k = 10):  145857
move count (k = 11):  134423
move count (k = 12):  138082
move count (k = 13):  143862


## feature vector 변화 측정
- 클러스터가 바꼈을 때 변수가 얼마나 변했는지 보기
  - e.g., 어떤 그룹이 클러스터 7→10 바꼈다면, 클러스터 7이었을 때와 클러스터 10이었을 때의 각각 소비 비율 vector 있을 것. 둘의 유클리디언 거리나 절댓값 합의 차이를 통해 변화량 측정
- 클러스터 개수가 많아질수록 클러스터가 서로 비슷해지니까, 변화량 작아지는 트렌드 (>>> 클러스터 변화가 많이 일어나서 변화량 커지는거 아닌가 ??). 
  - 이 변화량이 덜 작아지는 구간 찾기. 
- 클러스터 개수 많아질수록 어떤 변수들에서 점점 차이가 보이는지 분석할 수 있으면 좋을 듯
---
- feature 18개라서 애매함 (뭐가 중요한 feature인지 파악 불가)
---
- 코드 작동 방식
  - 1. 한 고객 그룹의 월별 클러스터 membership 보기
      - [1,1,1,1,2,3,1,1,1]
  - 2. 클러스터가 변한 경우, 각 feature의 변화량 구하기
      - 음식점 [       0.17, -0.2, 0,1          ] 
      - 병원 [       0.11, -0.11, 0,71          ]
  - 3. 변화량 절댓값 평균 구하기
      - 음식점 0.47 / 3 = 0.15
      - 병원 0.93 / 3 = 0.31
  - 4. 모든 고객에 대하여 1~3 구한 후 평균 내기
- household heterogenity 연구에서는 정규화 위해 %변화량 구했지만, 카드 데이터에서는 모든 feature들이 0~1로 이미 정규화되어있으므로 그냥 변화량 구함

In [ ]:
final_list = []

for df in tqdm(all_data_k):

  df = df.sort_values(by=['년','월'])
  grouped = df.groupby(by=['가맹점소재지1','가맹점소재지2','성별','연령대별','연평균소득추정'])

  """ transition_dict 생성 """
  transition_dict = {}
  for key in grouped.groups.keys():
    transition_dict[key] = list(grouped.get_group(key)['cluster'])

  dict_for_k = {}
  for feature in df.columns[7:25]: # ['가정용품', '개인미용', ..., '종합소매', '주거']

    """ values_dict 생성 """
    values_dict = {}
    for key in grouped.groups.keys():
      values_dict[key] = list(grouped.get_group(key)[feature])

    """ feature _list 생성 """
    feature_list = []

    for key in grouped.groups.keys(): # ('강원도', '강릉시', '남성', '20대', 'B1')

      """ transition_dict """
      item = transition_dict[key] # [4, 3, 1, 1, 1]

      if np.sum(np.diff(item)) == 0: ### 클러스터 변화가 한 번도 없는 경우
        continue

      temp_list = np.diff(item)
      index_list = list(np.nonzero(temp_list)[0]) # 클러스터 변화가 일어난 위치, [0, 1]

      """ values_dict """
      item = values_dict[key] # [0.01, 0.04, 0.09, 0.09, 0.19]
      temp_list = np.diff(item)
      diff_list = temp_list[index_list] #  클러스터 변화가 일어났을 때의 diff, [0.03, 0.05]

      sum = np.sum(np.abs(diff_list)) # 0.08
      avg = sum / len(diff_list) # 0.04

      feature_list.append(avg)
    
    dict_for_k[feature] = feature_list

  final_list.append(list(pd.DataFrame(dict_for_k).mean().values))




result = pd.DataFrame(final_list, columns=['가정용품', '개인미용',
                                          '개인운송', '교육서비스', '기타', '대형판매', '보건', '보험', '숙박서비스', '식료품', '여객운송',
                                          '오락_문화', '음식점', '의류_잡화', '전자상거래', '정보통신', '종합소매', '주거'],
                                            index = k_list)
result['count'] = count_list
result.to_csv(data_dir + 'avg_abs_diff_ver{0}_outliered.csv'.format(ver))
result

100%|██████████| 7/7 [18:26<00:00, 158.04s/it]


,가정용품,개인미용,개인운송,교육서비스,기타,대형판매,보건,보험,숙박서비스,식료품,여객운송,오락_문화,음식점,의류_잡화,전자상거래,정보통신,종합소매,주거,count
7,32406.288880,12589.250836,13850.666871,38309.644589,15515.406012,12134.498473,12711.485306,7043.326380,15764.164302,7069.914848,9606.096513,16128.000588,7943.308141,8731.482266,34103.564154,4310.437730,3613.119463,9108.197842,113328
8,37105.389620,14904.300262,16392.213021,34797.628877,18794.625056,11568.396082,15377.488717,7297.708103,18247.852584,8449.155329,10137.502182,22006.330988,7352.413623,9644.150670,22677.753214,3952.103105,3681.508616,14644.954759,119760
9,36001.549099,15438.952329,15307.168282,37992.714622,16243.873977,10913.859648,15257.697625,7215.615753,16153.933063,7249.301405,11772.309711,19870.073457,7787.288537,9161.721397,23776.786066,5494.485709,3342.671962,10165.444185,126871
10,37370.561716,13541.308629,13835.274810,33495.197283,17780.395984,10220.641939,13954.440352,6541.633887,17292.395118,7702.118051,7506.678973,18588.706192,7698.539840,8260.452745,20732.121226,5099.490888,3486.119360,12914.317749,145857
11,37647.602231,13005.340502,16310.771311,34988.811542,16700.729661,10648.392644,15152.826452,7587.333642,17864.896857,7501.615913,11228.786536,20539.101190,7475.855969,8563.134287,23486.213714,5135.543824,3390.631610,14283.587874,134423
12,33019.238342,15028.293164,14136.545808,35639.361127,17109.884671,11531.067344,16143.098906,6773.247632,16769.401180,7666.889162,9706.749238,19668.609167,7537.181609,8247.163689,28938.342625,4973.418528,3396.994453,14454.248238,138082
13,38127.509112,15064.505625,14871.198987,33370.581630,16385.359961,11774.757061,14074.185465,8241.197024,17949.623497,7938.046167,9917.183105,21380.433754,7582.682614,8896.215074,27456.438511,5558.418601,3400.763172,11695.634320,143862


In [ ]:
result = pd.read_csv(data_dir + 'avg_abs_diff_ver{0}_outliered.csv'.format(ver))

In [ ]:
# 정렬
result = result[['count', '음식점', '종합소매', '개인운송', '대형판매', '오락_문화', '보건', '식료품', '전자상거래', '교육서비스',
                  '여객운송', '의류_잡화', '숙박서비스', '개인미용', '주거', '기타', '가정용품', '보험', '정보통신']]
# index
result['k'] = k_list
result = result.set_index('k')

import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
result = result.style.background_gradient(cmap=cm)
display(result)

,count,음식점,종합소매,개인운송,대형판매,오락_문화,보건,식료품,전자상거래,교육서비스,여객운송,의류_잡화,숙박서비스,개인미용,주거,기타,가정용품,보험,정보통신
k,,,,,,,,,,,,,,,,,,,
7,113328,7943.308141,3613.119463,13850.666871,12134.498473,16128.000588,12711.485306,7069.914848,34103.564154,38309.644589,9606.096513,8731.482266,15764.164302,12589.250836,9108.197842,15515.406012,32406.288880,7043.326380,4310.437730
8,119760,7352.413623,3681.508616,16392.213021,11568.396082,22006.330988,15377.488717,8449.155329,22677.753214,34797.628877,10137.502182,9644.150670,18247.852584,14904.300262,14644.954759,18794.625056,37105.389620,7297.708103,3952.103105
9,126871,7787.288537,3342.671962,15307.168282,10913.859648,19870.073457,15257.697625,7249.301405,23776.786066,37992.714622,11772.309711,9161.721397,16153.933063,15438.952329,10165.444185,16243.873977,36001.549099,7215.615753,5494.485709
10,145857,7698.539840,3486.119360,13835.274810,10220.641939,18588.706192,13954.440352,7702.118051,20732.121226,33495.197283,7506.678973,8260.452745,17292.395118,13541.308629,12914.317749,17780.395984,37370.561716,6541.633887,5099.490888
11,134423,7475.855969,3390.631610,16310.771311,10648.392644,20539.101190,15152.826452,7501.615913,23486.213714,34988.811542,11228.786536,8563.134287,17864.896857,13005.340502,14283.587874,16700.729661,37647.602231,7587.333642,5135.543824
12,138082,7537.181609,3396.994453,14136.545808,11531.067344,19668.609167,16143.098906,7666.889162,28938.342625,35639.361127,9706.749238,8247.163689,16769.401180,15028.293164,14454.248238,17109.884671,33019.238342,6773.247632,4973.418528
13,143862,7582.682614,3400.763172,14871.198987,11774.757061,21380.433754,14074.185465,7938.046167,27456.438511,33370.581630,9917.183105,8896.215074,17949.623497,15064.505625,11695.634320,16385.359961,38127.509112,8241.197024,5558.418601
